# Introduction/Business Problem
## 1. A description of the problem and a discussion of the background

In this project I will compare neiberhoods of two big cities in Europe, Amsterdam and Berlin.

My target audience are people that know one city pretty well and are wondering which neighborhood of the other city is most similar to the one they like.

They could use this data to choose location of their hotel or apartment when visiting or moving to the other city. 

Criteria that I will take into the consideration are, Population density, area, venues nearby

# Data 
## 2. A description of the data and how it will be used to solve the problem. 

I will take postal codes, neigboorhood info, geo location from, **https://postal-codes.cybo.com/germany/berlin/#listcodes**, **https://postal-codes.cybo.com/netherlands/amsterdam/#listcodes** then using the foursquare API, I will obtain data about the venues in each neiborhood to compare them.

The details page on **cybo.com** have **Neighborhoods**, **Coordinates**, **area** and **population** fields 
Foursquare endpoint gives information about the venues **https://api.foursquare.com/v2/venues/explore**

First entry from the Berlin codes:


Postal Code|City|Administrative Region|Population|Area
 --- | --- | --- | --- | --- 
10115|Berlin|Berlin|16,678|2.421 km²

And in the details page **https://postal-codes.cybo.com/germany/10115_berlin-berlin/** fields that I will use:

| | |
:--- | --- 
**Neighborhoods**|Mitte
**Coordinates**|52.533706954560394° / 13.387223860255002°

Interesing information from the API are in the items, especialy venue fields.
Foursquare API result for the above coords, one venue (Berlin Wall Memorial):
```
{
	"meta": {
		"code": 200,
		"requestId": "6106e77db4ece17b334c5722"
	},
	"notifications": [
		{
			"type": "notificationTray",
			"item": {
				"unreadCount": 0
			}
		}
	],
	"response": {
		"queryRefinements": {
			"target": {
				"type": "path",
				"url": "/venue/explore",
				"params": {
					"ll": "52.533707,13.387224",
					"radius": "500"
				}
			},
			"refinements": [
				{
					"query": "Food"
				},
				{
					"query": "Nightlife"
				},
				{
					"query": "Coffee"
				},
				{
					"query": "Shops"
				},
				{
					"query": "Arts"
				},
				{
					"query": "Outdoors"
				}
			]
		},
		"suggestedFilters": {
			"header": "Tap to show:",
			"filters": [
				{
					"name": "Open now",
					"key": "openNow"
				}
			]
		},
		"warning": {
			"text": "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."
		},
		"headerLocation": "Gesundbrunnen",
		"headerFullLocation": "Gesundbrunnen, Berlin",
		"headerLocationGranularity": "neighborhood",
		"totalResults": 72,
		"suggestedBounds": {
			"ne": {
				"lat": 52.538206959060399,
				"lng": 13.394607786572898
			},
			"sw": {
				"lat": 52.52920695006039,
				"lng": 13.379839933937106
			}
		},
		"groups": [
			{
				"type": "Recommended Places",
				"name": "recommended",
				"items": [
					{
						"reasons": {
							"count": 0,
							"items": [
								{
									"summary": "This spot is popular",
									"pluralSummary": "These spots are popular",
									"type": "general",
									"reasonName": "globalInteractionReason"
								}
							]
						},
						"snippets": {
							"count": 1,
							"items": [
								{
									"detail": {
										"type": "tip",
										"object": {
											"id": "4e204721922e11c006640fc5",
											"createdAt": 1310738209,
											"text": "See some original remains of the Berlin Wall. Watch the documentary movies at the visitors centre.",
											"type": "user",
											"canonicalUrl": "https://foursquare.com/item/4e204721922e11c006640fc5",
											"canonicalPath": "/item/4e204721922e11c006640fc5",
											"likes": {
												"count": 10,
												"groups": [],
												"summary": "10 likes"
											},
											"logView": true,
											"agreeCount": 8,
											"disagreeCount": 0,
											"todo": {
												"count": 8
											},
											"user": {
												"id": "9407435",
												"firstName": "Best Western Central Europe",
												"gender": "none",
												"countryCode": "DE",
												"canonicalUrl": "https://foursquare.com/p/best-western-central-europe/9407435",
												"canonicalPath": "/bestwesternce",
												"photo": {
													"prefix": "https://fastly.4sqi.net/img/user/",
													"suffix": "/9407435-TNBWV23T4AF30XMX.png"
												},
												"type": "chain",
												"isAnonymous": false
											}
										}
									}
								}
							]
						},
						"venue": {
							"id": "4bd975242e6f0f4708960a08",
							"name": "Berlin Wall Memorial",
							"canonicalName": "Gedenkstätte Berliner Mauer",
							"contact": {
								"phone": "+4930467986666",
								"formattedPhone": "+49 30 467986666",
								"facebook": "119701714673",
								"facebookUsername": "Stiftung.Berliner.Mauer",
								"facebookName": "Stiftung Berliner Mauer"
							},
							"location": {
								"address": "Bernauer Str. 111",
								"crossStreet": "Ackerstr.",
								"lat": 52.534896052519219,
								"lng": 13.390139937400818,
								"labeledLatLngs": [
									{
										"label": "display",
										"lat": 52.534896052519219,
										"lng": 13.390139937400818
									}
								],
								"distance": 237,
								"postalCode": "13355",
								"cc": "DE",
								"neighborhood": "Mitte",
								"city": "Berlin",
								"state": "Berlin",
								"country": "Niemcy",
								"contextLine": "Mitte",
								"contextGeoId": 65337,
								"formattedAddress": [
									"<span itemprop=\"streetAddress\">Bernauer Str. 111</span> (Ackerstr.)",
									"<span itemprop=\"postalCode\">13355</span> <span itemprop=\"addressLocality\">Berlin</span>",
									"Niemcy"
								]
							},
							"canonicalUrl": "https://foursquare.com/v/gedenkst%C3%A4tte-berliner-mauer/4bd975242e6f0f4708960a08",
							"canonicalPath": "/v/gedenkst%C3%A4tte-berliner-mauer/4bd975242e6f0f4708960a08",
							"categories": [
								{
									"id": "4deefb944765f83613cdba6e",
									"name": "Historic Site",
									"pluralName": "Historic Sites",
									"shortName": "Historic Site",
									"icon": {
										"prefix": "https://ss3.4sqi.net/img/categories_v2/arts_entertainment/historicsite_",
										"mapPrefix": "https://ss3.4sqi.net/img/categories_map/arts_entertainment/museum",
										"suffix": ".png"
									},
									"primary": true
								}
							],
							"verified": false,
							"stats": {
								"tipCount": 196,
								"usersCount": 16284,
								"checkinsCount": 19247
							},
							"url": "http://www.berliner-mauer-gedenkstaette.de/",
							"urlSig": "eHOdBlfOHxAr0NOEPkiipbv2eM0=",
							"rating": 8.9,
							"ratingColor": "73CF42",
							"ratingSignals": 1571,
							"allowMenuUrlEdit": true,
							"beenHere": {
								"count": 0,
								"marked": false,
								"lastCheckinExpiredAt": 0
							},
							"hours": {
								"status": "Open until 10:00 PM",
								"richStatus": {
									"entities": [],
									"text": "Open until 10:00 PM"
								},
								"isOpen": true,
								"isLocalHoliday": false
							},
							"photos": {
								"count": 0,
								"groups": []
							},
							"hereNow": {
								"count": 0,
								"summary": "Nobody here",
								"groups": []
							},
							"popularityByGeo": 0.9992924061022898
						},
						"photo": {
							"id": "501e8b5ce4b0d64f431c46da",
							"createdAt": 1344179036,
							"prefix": "https://fastly.4sqi.net/img/general/",
							"suffix": "/AD7POWLlLxpzJS_ja9X4kdU1O5U3jtRJR-xQ4xjRi5I.jpg",
							"width": 720,
							"height": 537,
							"visibility": "public"
						},
						"tips": [
							{
								"id": "4e204721922e11c006640fc5",
								"createdAt": 1310738209,
								"text": "See some original remains of the Berlin Wall. Watch the documentary movies at the visitors centre.",
								"type": "user",
								"canonicalUrl": "https://foursquare.com/item/4e204721922e11c006640fc5",
								"canonicalPath": "/item/4e204721922e11c006640fc5",
								"likes": {
									"count": 10,
									"groups": [],
									"summary": "10 likes"
								},
								"logView": true,
								"agreeCount": 8,
								"disagreeCount": 0,
								"todo": {
									"count": 8
								},
								"user": {
									"id": "9407435",
									"firstName": "Best Western Central Europe",
									"gender": "none",
									"countryCode": "DE",
									"canonicalUrl": "https://foursquare.com/p/best-western-central-europe/9407435",
									"canonicalPath": "/bestwesternce",
									"photo": {
										"prefix": "https://fastly.4sqi.net/img/user/",
										"suffix": "/9407435-TNBWV23T4AF30XMX.png"
									},
									"type": "chain",
									"isAnonymous": false
								}
							}
						],
						"referralId": "e-0-4bd975242e6f0f4708960a08-0"
					}
				]
			}
		]
	}
}
```